# Open double-funded channel without intermediary

In [1]:
%load_ext autoreload
%autoreload 2

## Setup LND clients for Ali and Bob

In [2]:
import sys

In [3]:
# LND GRPC bindings should be in '/home/jovyan/lnrpc'
sys.path.append('/home/jovyan/lnrpc')

In [4]:
%%sh
sudo cp /ali-lnd/data/chain/bitcoin/regtest/admin.macaroon /tmp/ali.macaroon
sudo chmod +r /tmp/ali.macaroon

sudo cp /bob-lnd/data/chain/bitcoin/regtest/admin.macaroon /tmp/bob.macaroon
sudo chmod +r /tmp/bob.macaroon

In [5]:
from p2oc.lnd_rpc import LndRpc, lnmsg, walletmsg, signrpc, signmsg, routermsg

In [6]:
ali = LndRpc(host='ali-lnd:10009',
             cert_path='/ali-lnd/tls.cert',
             macaroon_path='/tmp/ali.macaroon')

In [7]:
bob = LndRpc(host='bob-lnd:10009',
             cert_path='/bob-lnd/tls.cert',
             macaroon_path='/tmp/bob.macaroon')

In [8]:
aliNodePubKey = ali.lnd.GetInfo(lnmsg.GetInfoRequest()).identity_pubkey
aliNodePubKey

'0304503caa78643f20487cb225a2379d6222f279666f58e0fa50c81bf8e01122e1'

In [9]:
bobNodePubKey = bob.lnd.GetInfo(lnmsg.GetInfoRequest()).identity_pubkey
bobNodePubKey

'029e06e3586e8de2e970c16d389b22c9a2d1beacdb2cd2d900472334c4a9cd8908'

## Fund Ali and Bob

In [15]:
import os
import bitcoin
from p2oc.btc_rpc import Proxy, Config

In [16]:
bitcoin.SelectParams('regtest')

In [17]:
brpc = Proxy(config=Config(
    rpcuser=os.environ['BTCD_RPCUSER'],
    rpcpassword=os.environ['BTCD_RPCPASS'],
    rpcconnect='bitcoind',
    rpcport=18443
))

In [18]:
brpc.getblockcount()

0

In [19]:
aliAddr = ali.lnd.NewAddress(lnmsg.NewAddressRequest(type=0)) # p2wkh
aliAddr

address: "bcrt1q923ygfsttuyqmze53qranv0zs4j7z00ulatthe"

In [20]:
bobAddr = bob.lnd.NewAddress(lnmsg.NewAddressRequest(type=0)) # p2wkh
bobAddr

address: "bcrt1q2rvvl76fs3t5kav75cvssn7lq0lf9jz4pla2eq"

In [21]:
# fund ali
_ = list(brpc.generatetoaddress(1, aliAddr.address))

In [22]:
# fund bob
_ = list(brpc.generatetoaddress(1, bobAddr.address))

In [23]:
brpc.createwallet('miner')
minerAddr = brpc.getnewaddress("coinbase")
brpc.unloadwallet('miner')

{'warning': ''}

In [24]:
# unlock mined coins
_ = list(brpc.generatetoaddress(110, minerAddr))

In [25]:
ali.lnd.WalletBalance(lnmsg.WalletBalanceRequest())

total_balance: 5000000000
confirmed_balance: 5000000000
account_balance {
  key: "default"
  value {
    confirmed_balance: 5000000000
  }
}

In [26]:
bob.lnd.WalletBalance(lnmsg.WalletBalanceRequest())

total_balance: 5000000000
confirmed_balance: 5000000000
account_balance {
  key: "default"
  value {
    confirmed_balance: 5000000000
  }
}

## Ali creates public offer to request inbound liquidity in exchange for a fee

In [27]:
import hashlib
import time
import base64

import bitcoin.core as bc
import bitcoin.core.script as bs
import bitcoin.wallet as bw
from bitcoin.rpc import hexlify, unhexlify

In [28]:
# taker needs to pay premium to open channel
premiumAmount = int(0.001 * bc.COIN) # premium Ali is willing to pay
fundAmount = int(0.16 * bc.COIN) # requested inbound capacity

In [29]:
# create dummy psbt to extract 'funding' UTXO and change addresses
# user minerAddr as a dummy address to create psbt
tx_template = walletmsg.TxTemplate(outputs={str(minerAddr): premiumAmount})

psbtRequest = walletmsg.FundPsbtRequest(raw=tx_template, target_conf=6)

In [30]:
aliDummyPsbt = ali.wallet.FundPsbt(request=psbtRequest)

In [31]:
psbtBase64 = base64.b64encode(aliDummyPsbt.funded_psbt).decode()

In [32]:
aliDummyPsbtDecoded = brpc._proxy._call('decodepsbt', psbtBase64)
aliDummyPsbtDecoded

{'tx': {'txid': '130fc0f350bcdfb46e944158a7f3514bbfc73e4cff8fa9cf153a2801c2c70937',
  'hash': '130fc0f350bcdfb46e944158a7f3514bbfc73e4cff8fa9cf153a2801c2c70937',
  'version': 2,
  'size': 113,
  'vsize': 113,
  'weight': 452,
  'locktime': 0,
  'vin': [{'txid': '0c29d7d862b3b670f79d7f57eda0a512332f96e68a377535e70c14b82582001d',
    'vout': 0,
    'scriptSig': {'asm': '', 'hex': ''},
    'sequence': 4294967295}],
  'vout': [{'value': Decimal('0.00100000'),
    'n': 0,
    'scriptPubKey': {'asm': '0 eac1ce29853c2ec4a95c8ed3cf36743bdc659353',
     'hex': '0014eac1ce29853c2ec4a95c8ed3cf36743bdc659353',
     'reqSigs': 1,
     'type': 'witness_v0_keyhash',
     'addresses': ['bcrt1qatquu2v98shvf22u3mfu7dn580wxty6nwjq305']}},
   {'value': Decimal('49.99892950'),
    'n': 1,
    'scriptPubKey': {'asm': '0 c8b6f8f6278d569194004ba1def4717803d753be',
     'hex': '0014c8b6f8f6278d569194004ba1def4717803d753be',
     'reqSigs': 1,
     'type': 'witness_v0_keyhash',
     'addresses': ['bcrt1qezm03a3

In [33]:
aliInputs = []
for vin in aliDummyPsbtDecoded['tx']['vin']:
    txin = bc.CTxIn(bc.COutPoint(bc.lx(vin['txid']), vin['vout']))
    aliInputs.append(txin)
aliInputs

[CTxIn(COutPoint(lx('0c29d7d862b3b670f79d7f57eda0a512332f96e68a377535e70c14b82582001d'), 0), CScript([]), 0xffffffff)]

In [34]:
# assume there are only 2 outputs: change and channel funding
assert len(aliDummyPsbtDecoded['tx']['vout']) == 2

In [35]:
aliChangeOutput = aliDummyPsbtDecoded['tx']['vout'][aliDummyPsbt.change_output_index]
aliChangeOutput = bc.CTxOut(int(aliChangeOutput['value'] * bc.COIN),
                            bc.CScript(unhexlify(aliChangeOutput['scriptPubKey']['hex'])))

aliChangeOutput

CTxOut(49.9989295*COIN, CScript([0, x('c8b6f8f6278d569194004ba1def4717803d753be')]))

In [36]:
# Generate pubkey for channel funding
key_family = 0 # multisig?
keyReq = walletmsg.KeyReq(key_family=key_family)
aliKeyDesc = ali.wallet.DeriveNextKey(keyReq)
aliKeyDesc = ali.wallet.DeriveNextKey(keyReq) # need to call twice?
aliKeyDesc

raw_key_bytes: "\002\023\261\340j\177QC/\236\363o\337\253\216\365\370\306\203\202\361\004\362\225zP\272\371\260\353\366\273\346"
key_loc {
  key_index: 1
}

In [37]:
offer = {
    'id': hashlib.sha256(str(time.time()).encode()).hexdigest(), # some random id to match offers and replies
    'type': 'INBOUND_LIQUIDITY_REQUEST',
    'nodeAddr': 'ali-lnd',
    'nodePubKey': aliNodePubKey,
    'premiumAmount': premiumAmount,
    'fundAmount': fundAmount,
    'inputs': aliInputs,
    'change': aliChangeOutput,
    'chanPubKey1': aliKeyDesc.raw_key_bytes
}
offer

{'id': 'c0bdd96e3bbd2e4b4575bf465fbd975e3753dbd4cd4923870828689e171a3868',
 'type': 'INBOUND_LIQUIDITY_REQUEST',
 'nodeAddr': 'ali-lnd',
 'nodePubKey': '0304503caa78643f20487cb225a2379d6222f279666f58e0fa50c81bf8e01122e1',
 'premiumAmount': 100000,
 'fundAmount': 16000000,
 'inputs': [CTxIn(COutPoint(lx('0c29d7d862b3b670f79d7f57eda0a512332f96e68a377535e70c14b82582001d'), 0), CScript([]), 0xffffffff)],
 'change': CTxOut(49.9989295*COIN, CScript([0, x('c8b6f8f6278d569194004ba1def4717803d753be')])),
 'chanPubKey1': b'\x02\x13\xb1\xe0j\x7fQC/\x9e\xf3o\xdf\xab\x8e\xf5\xf8\xc6\x83\x82\xf1\x04\xf2\x95zP\xba\xf9\xb0\xeb\xf6\xbb\xe6'}

## Bob accepts offer and sends reply

### Check the the offer is still valid

In [38]:
# for simplicity assume single input
# check that utxo has not been spent
utxo = brpc.gettxout(offer['inputs'][0].prevout)
assert utxo is not None
utxo

{'bestblock': b' \x1c\xf6I<\x01\x18\x83\xd7"\xe6 \x0b\xb1G\xf6\rr#5\xd4{\x02|y>\xd7\x87\xe4\xffx\x06',
 'confirmations': 112,
 'coinbase': True,
 'txout': CTxOut(50.0*COIN, CScript([0, x('2aa244260b5f080d8b348807d9b1e28565e13dfc')]))}

In [39]:
# check that there is enough premium
inValue = utxo['txout'].nValue
changeValue = offer['change'].nValue

# fees can be split between maker and taker
fees = inValue - offer['premiumAmount'] - changeValue
assert fees > 0
fees

7050

### Allocate funds

In [40]:
# for simplicity Bob is maker but it can take "taker" role as well
assert offer['type'] == 'INBOUND_LIQUIDITY_REQUEST'

In [41]:
premiumAmount = offer['premiumAmount'] # Ali's premium to Bob
fundAmount = offer['fundAmount'] # Bob funds this amount

In [42]:
# create dummy psbt to extract 'funding' UTXO and change addresses
# user minerAddr as a dummy address to create psbt
tx_template = walletmsg.TxTemplate(outputs={str(minerAddr): fundAmount})

psbtRequest = walletmsg.FundPsbtRequest(raw=tx_template, target_conf=6)
psbtRequest

raw {
  outputs {
    key: "bcrt1qatquu2v98shvf22u3mfu7dn580wxty6nwjq305"
    value: 16000000
  }
}
target_conf: 6

In [43]:
bobDummyPsbt = bob.wallet.FundPsbt(request=psbtRequest)

In [44]:
psbtBase64 = base64.b64encode(bobDummyPsbt.funded_psbt).decode()

In [45]:
bobDummyPsbtDecoded = brpc._proxy._call('decodepsbt', psbtBase64)
bobDummyPsbtDecoded

{'tx': {'txid': '95b071af89073df40e7278e042ad6eceb7dd432a867cb5948e3565b5dd3eeeea',
  'hash': '95b071af89073df40e7278e042ad6eceb7dd432a867cb5948e3565b5dd3eeeea',
  'version': 2,
  'size': 113,
  'vsize': 113,
  'weight': 452,
  'locktime': 0,
  'vin': [{'txid': '18312e0b38fc404be7ac9c23303d05b4a4c303d8e19e6d9d665cdef5a162a340',
    'vout': 0,
    'scriptSig': {'asm': '', 'hex': ''},
    'sequence': 4294967295}],
  'vout': [{'value': Decimal('0.16000000'),
    'n': 0,
    'scriptPubKey': {'asm': '0 eac1ce29853c2ec4a95c8ed3cf36743bdc659353',
     'hex': '0014eac1ce29853c2ec4a95c8ed3cf36743bdc659353',
     'reqSigs': 1,
     'type': 'witness_v0_keyhash',
     'addresses': ['bcrt1qatquu2v98shvf22u3mfu7dn580wxty6nwjq305']}},
   {'value': Decimal('49.83992950'),
    'n': 1,
    'scriptPubKey': {'asm': '0 34086da2b8920974cdd10f4ab371da0457e12678',
     'hex': '001434086da2b8920974cdd10f4ab371da0457e12678',
     'reqSigs': 1,
     'type': 'witness_v0_keyhash',
     'addresses': ['bcrt1qxsyxmg4

In [46]:
bobInputs = []
for vin in bobDummyPsbtDecoded['tx']['vin']:
    txin = bc.CTxIn(bc.COutPoint(bc.lx(vin['txid']), vin['vout']))
    bobInputs.append(txin)
bobInputs

[CTxIn(COutPoint(lx('18312e0b38fc404be7ac9c23303d05b4a4c303d8e19e6d9d665cdef5a162a340'), 0), CScript([]), 0xffffffff)]

In [47]:
# assume there are only 2 outputs: change and channel funding
assert len(bobDummyPsbtDecoded['tx']['vout']) == 2

In [48]:
# NOTE: bob pays funding tx fee but he does not have to
bobChangeOutput = bobDummyPsbtDecoded['tx']['vout'][bobDummyPsbt.change_output_index]
bobChangeOutput = bc.CTxOut(int(bobChangeOutput['value'] * bc.COIN),
                            bc.CScript(unhexlify(bobChangeOutput['scriptPubKey']['hex'])))

bobChangeOutput

CTxOut(49.8399295*COIN, CScript([0, x('34086da2b8920974cdd10f4ab371da0457e12678')]))

### Create funding transaction

In [49]:
key_family = 0 # multisig?
keyReq = walletmsg.KeyReq(key_family=key_family)
bobKeyDesc = bob.wallet.DeriveNextKey(keyReq)
bobKeyDesc = bob.wallet.DeriveNextKey(keyReq) # need to call twice?
bobKeyDesc

raw_key_bytes: "\002\262\236\351\345\355\334\240\36344\317dT\333\342\276\313\342\206\2713?\372\233\200\037\022\266\337\357)\227"
key_loc {
  key_index: 1
}

In [50]:
takerPubKey = offer['chanPubKey1']
makerPubKey = bobKeyDesc.raw_key_bytes

In [51]:
# https://github.com/lightningnetwork/lightning-rfc/blob/master/03-transactions.md#funding-transaction-output
# https://github.com/bitcoin/bips/blob/master/bip-0069.mediawiki
# if makerPubKey > takerPubKey:
if list(makerPubKey) > list(takerPubKey):
    pk1, pk2 = takerPubKey, makerPubKey
else:
    pk1, pk2 = makerPubKey, takerPubKey

In [52]:
msigScript = bs.CScript([
        bs.OP_2,
        pk1,
        pk2,
        bs.OP_2,
        bs.OP_CHECKMULTISIG
    ])

In [53]:
# convert to P2WSH
scriptPubKey = bc.CScript([bs.OP_0, hashlib.sha256(msigScript).digest()])
scriptPubKey

CScript([0, x('eebd4b5e50d80a16388c9d1ec0db26047afba0f724c1283f17068ec9f265afab')])

In [54]:
assert scriptPubKey.is_witness_v0_scripthash()

In [55]:
fundingOutput = bc.CTxOut(premiumAmount + fundAmount,
                          scriptPubKey)
fundingOutput

CTxOut(0.161*COIN, CScript([0, x('eebd4b5e50d80a16388c9d1ec0db26047afba0f724c1283f17068ec9f265afab')]))

In [56]:
# for simplicity Bob's input is at position 0
fundingTxInputs = bobInputs + offer['inputs']
fundingTxOutputs = [fundingOutput, bobChangeOutput, offer['change']]
fundingOutputIdx = 0

In [57]:
fundingTx = bc.CTransaction(fundingTxInputs, fundingTxOutputs)
fundingTx

CTransaction((CTxIn(COutPoint(lx('18312e0b38fc404be7ac9c23303d05b4a4c303d8e19e6d9d665cdef5a162a340'), 0), CScript([]), 0xffffffff), CTxIn(COutPoint(lx('0c29d7d862b3b670f79d7f57eda0a512332f96e68a377535e70c14b82582001d'), 0), CScript([]), 0xffffffff)), (CTxOut(0.161*COIN, CScript([0, x('eebd4b5e50d80a16388c9d1ec0db26047afba0f724c1283f17068ec9f265afab')])), CTxOut(49.8399295*COIN, CScript([0, x('34086da2b8920974cdd10f4ab371da0457e12678')])), CTxOut(49.9989295*COIN, CScript([0, x('c8b6f8f6278d569194004ba1def4717803d753be')]))), 0, 1, CTxWitness())

In [58]:
fundingTxId = fundingTx.GetTxid()
fundingTxId

b'\xb3\xb3\xff\xdaH\x17Yl\x1dg\xd0{^z8\xd8\x96\xa7\xd9\xfa\xe9\x8bDC\x15kDx=\xb0\xea\xb8'

### connect to taker (Ali)

In [59]:
# assumer whoever creates offer is accessible
connect_peer_req = lnmsg.ConnectPeerRequest(
    addr=lnmsg.LightningAddress(
        pubkey=offer['nodePubKey'],
        host=offer['nodeAddr']
))

In [60]:
bob.lnd.ConnectPeer(connect_peer_req)

In [61]:
# check that we are connected
bob.lnd.ListPeers(lnmsg.ListPeersRequest())

peers {
  pub_key: "0304503caa78643f20487cb225a2379d6222f279666f58e0fa50c81bf8e01122e1"
  address: "172.19.0.3:9735"
  bytes_sent: 142
  bytes_recv: 142
  sync_type: ACTIVE_SYNC
  features {
    key: 0
    value {
      name: "data-loss-protect"
      is_required: true
      is_known: true
    }
  }
  features {
    key: 5
    value {
      name: "upfront-shutdown-script"
      is_known: true
    }
  }
  features {
    key: 7
    value {
      name: "gossip-queries"
      is_known: true
    }
  }
  features {
    key: 9
    value {
      name: "tlv-onion"
      is_known: true
    }
  }
  features {
    key: 12
    value {
      name: "static-remote-key"
      is_required: true
      is_known: true
    }
  }
  features {
    key: 14
    value {
      name: "payment-addr"
      is_required: true
      is_known: true
    }
  }
  features {
    key: 17
    value {
      name: "multi-path-payments"
      is_known: true
    }
  }
  features {
    key: 31
    value {
      name: "amp"
      i

### Register shim to prepare for channel opening

In [62]:
# something that is unique to the corresponding matched taker and maker
channelId = hashlib.sha256((bobNodePubKey + offer['nodePubKey'] + str(time.time())).encode()).hexdigest()
channelId

'687cea81ef36ce5d2deda090171bda2e733e31568fd04e2cf74d54bfe0753036'

In [63]:
chan_point = lnmsg.ChannelPoint(funding_txid_bytes=fundingTxId,
                                output_index=fundingOutputIdx)

In [64]:
# define our key for funding output
local_key = lnmsg.KeyDescriptor(
    raw_key_bytes=bobKeyDesc.raw_key_bytes,
    key_loc=lnmsg.KeyLocator(
        key_family=bobKeyDesc.key_loc.key_family,
        key_index=bobKeyDesc.key_loc.key_index))

In [65]:
chan_point_shim = lnmsg.ChanPointShim(
    amt=fundAmount + premiumAmount,
    chan_point=chan_point,
    local_key=local_key,
    remote_key=offer['chanPubKey1'],
    pending_chan_id=unhexlify(channelId),
    thaw_height=0 # set 0 for simplicity
)

In [66]:
ftm = lnmsg.FundingTransitionMsg(
    shim_register=lnmsg.FundingShim(chan_point_shim=chan_point_shim))

In [67]:
# empty response
bob.lnd.FundingStateStep(ftm)

### Sign transaction

In [68]:
# At this point things look good and we can sign the transaction

In [69]:
# assume single input for simplicity
assert len(bobDummyPsbtDecoded['inputs']) == 1

In [70]:
input_ = bobDummyPsbtDecoded['inputs'][0]
input_

{'witness_utxo': {'amount': Decimal('50.00000000'),
  'scriptPubKey': {'asm': '0 50d8cffb4984574b759ea619084fdf03fe92c855',
   'hex': '001450d8cffb4984574b759ea619084fdf03fe92c855',
   'type': 'witness_v0_keyhash',
   'address': 'bcrt1q2rvvl76fs3t5kav75cvssn7lq0lf9jz4pla2eq'}},
 'non_witness_utxo': {'txid': '18312e0b38fc404be7ac9c23303d05b4a4c303d8e19e6d9d665cdef5a162a340',
  'hash': '0052ab4d20b74f7bb13827ffa777f0043245aa889b21eee499e922e2dfeb47d9',
  'version': 2,
  'size': 168,
  'vsize': 141,
  'weight': 564,
  'locktime': 0,
  'vin': [{'coinbase': '520101',
    'txinwitness': ['0000000000000000000000000000000000000000000000000000000000000000'],
    'sequence': 4294967295}],
  'vout': [{'value': Decimal('50.00000000'),
    'n': 0,
    'scriptPubKey': {'asm': '0 50d8cffb4984574b759ea619084fdf03fe92c855',
     'hex': '001450d8cffb4984574b759ea619084fdf03fe92c855',
     'reqSigs': 1,
     'type': 'witness_v0_keyhash',
     'addresses': ['bcrt1q2rvvl76fs3t5kav75cvssn7lq0lf9jz4pla2eq']}

In [71]:
# our UTXO pubkey
assert len(input_['bip32_derivs']) == 1
pubkey = input_['bip32_derivs'][0]['pubkey']
pubkey = unhexlify(pubkey)
pubkey

b'\x03\xdd\xef\xeb\x04\x18S\xa7?\xe4\xd7\x8e\x18\xe2\x8b\x0f\x8f/\xd83\xa2\xde\x86\xf1v\xf7sA|\xfdY\xb3\xd5'

In [72]:
scriptPubKey = input_['witness_utxo']['scriptPubKey']['hex']
scriptPubKey = bs.CScript(unhexlify(scriptPubKey))

utxoAddr = bw.CBitcoinAddress.from_scriptPubKey(scriptPubKey)
utxoAddr

P2WPKHBitcoinAddress('bcrt1q2rvvl76fs3t5kav75cvssn7lq0lf9jz4pla2eq')

In [73]:
sign_desc = signmsg.SignDescriptor(
    key_desc=signmsg.KeyDescriptor(
        raw_key_bytes=pubkey
    ),
    witness_script=utxoAddr.to_redeemScript(),
    output=signmsg.TxOut(
        value=int(input_['witness_utxo']['amount'] * bc.COIN),
        pk_script=utxoAddr.to_scriptPubKey()
    ),
    sighash=bs.SIGHASH_ALL, # so that transaction cannot be altered
    input_index=0 # for simplicity Bob's input is 0
)

In [74]:
signReq = signmsg.SignReq(
    raw_tx_bytes=fundingTx.serialize(),
    sign_descs=[sign_desc]
)

In [75]:
bobSignResp = bob.signer.SignOutputRaw(signReq)
bobSignResp

raw_sigs: "0D\002 (N\321nB\361\303\317Lo\367\246\247`\343\000s\306\3442w\307\360\255S\317l\010:_J\273\002 \024\236+\332\244\304\300.\317\223\256\334\224\0024\014\204\030\037\023,\334p\203a\004\n_\322eR\343"

In [76]:
signature = bobSignResp.raw_sigs[0] + bytes([bs.SIGHASH_ALL])

In [77]:
witness = [signature, pubkey]
bs.CScriptWitness(witness)

CScriptWitness(x('30440220284ed16e42f1c3cf4c6ff7a6a760e30073c6e43277c7f0ad53cf6c083a5f4abb0220149e2bdaa4c4c02ecf93aedc9402340c84181f132cdc708361040a5fd26552e301'),x('03ddefeb041853a73fe4d78e18e28b0f8f2fd833a2de86f176f773417cfd59b3d5'))

In [78]:
bobSignedFundingTx = bc.CTransaction(
    vin=fundingTx.vin,
    vout=fundingTx.vout,
    witness=bc.CTxWitness([bc.CTxInWitness(bs.CScriptWitness(witness))]))

### Send offer reply

In [79]:
offerReply = {
    'id': offer['id'],
    'nodeAddr': 'bob-lnd', # sending for extra check
    'nodePubKey': bobNodePubKey,
    'channelId': channelId,
    'chanPubKey2': bobKeyDesc.raw_key_bytes,
    'fundingOutputIdx': fundingOutputIdx,
    'fundingTx': bobSignedFundingTx
}
offerReply

{'id': 'c0bdd96e3bbd2e4b4575bf465fbd975e3753dbd4cd4923870828689e171a3868',
 'nodeAddr': 'bob-lnd',
 'nodePubKey': '029e06e3586e8de2e970c16d389b22c9a2d1beacdb2cd2d900472334c4a9cd8908',
 'channelId': '687cea81ef36ce5d2deda090171bda2e733e31568fd04e2cf74d54bfe0753036',
 'chanPubKey2': b'\x02\xb2\x9e\xe9\xe5\xed\xdc\xa0\xf344\xcfdT\xdb\xe2\xbe\xcb\xe2\x86\xb93?\xfa\x9b\x80\x1f\x12\xb6\xdf\xef)\x97',
 'fundingOutputIdx': 0,
 'fundingTx': CTransaction((CTxIn(COutPoint(lx('18312e0b38fc404be7ac9c23303d05b4a4c303d8e19e6d9d665cdef5a162a340'), 0), CScript([]), 0xffffffff), CTxIn(COutPoint(lx('0c29d7d862b3b670f79d7f57eda0a512332f96e68a377535e70c14b82582001d'), 0), CScript([]), 0xffffffff)), (CTxOut(0.161*COIN, CScript([0, x('eebd4b5e50d80a16388c9d1ec0db26047afba0f724c1283f17068ec9f265afab')])), CTxOut(49.8399295*COIN, CScript([0, x('34086da2b8920974cdd10f4ab371da0457e12678')])), CTxOut(49.9989295*COIN, CScript([0, x('c8b6f8f6278d569194004ba1def4717803d753be')]))), 0, 1, CTxWitness(CTxInWitness(CScr

## Ali checks offer reply and commits funding tx

### Check offer reply message

In [80]:
# check id
assert offer['id'] == offerReply['id']

In [81]:
# check connection
peers = ali.lnd.ListPeers(lnmsg.ListPeersRequest())
assert peers.peers[0].pub_key == offerReply['nodePubKey']
# we can also check `nodeAddr`

### Check transaction

In [82]:
tx = offerReply['fundingTx']

# check that our input was included. For simplicity assume we are the last
assert tx.vin[-1] == offer['inputs'][0]

# check that our change output was included
assert tx.vout[-1] == offer['change']

# check that signature is valid. For simplicity just check that it's present
assert tx.wit is not None

### Check funding output

In [83]:
premiumAmount = offer['premiumAmount']
fundAmount = offer['fundAmount']

In [84]:
takerPubKey = offer['chanPubKey1'] # ali
makerPubKey = offerReply['chanPubKey2'] # bob

In [85]:
# https://github.com/lightningnetwork/lightning-rfc/blob/master/03-transactions.md#funding-transaction-output
# https://github.com/bitcoin/bips/blob/master/bip-0069.mediawiki
if list(makerPubKey) > list(takerPubKey):
    pk1, pk2 = takerPubKey, makerPubKey
else:
    pk1, pk2 = makerPubKey, takerPubKey

In [86]:
msigScript = bs.CScript([
        bs.OP_2,
        pk1,
        pk2,
        bs.OP_2,
        bs.OP_CHECKMULTISIG
    ])

In [87]:
# convert to P2WSH
scriptPubKey = bc.CScript([bs.OP_0, hashlib.sha256(msigScript).digest()])
assert scriptPubKey.is_witness_v0_scripthash()

In [88]:
fundingOutput = bc.CTxOut(premiumAmount + fundAmount,
                          scriptPubKey)
assert fundingOutput == tx.vout[0]

### Open pending channel

In [89]:
offerReply['fundingTx']

CTransaction((CTxIn(COutPoint(lx('18312e0b38fc404be7ac9c23303d05b4a4c303d8e19e6d9d665cdef5a162a340'), 0), CScript([]), 0xffffffff), CTxIn(COutPoint(lx('0c29d7d862b3b670f79d7f57eda0a512332f96e68a377535e70c14b82582001d'), 0), CScript([]), 0xffffffff)), (CTxOut(0.161*COIN, CScript([0, x('eebd4b5e50d80a16388c9d1ec0db26047afba0f724c1283f17068ec9f265afab')])), CTxOut(49.8399295*COIN, CScript([0, x('34086da2b8920974cdd10f4ab371da0457e12678')])), CTxOut(49.9989295*COIN, CScript([0, x('c8b6f8f6278d569194004ba1def4717803d753be')]))), 0, 1, CTxWitness(CTxInWitness(CScriptWitness(x('30440220284ed16e42f1c3cf4c6ff7a6a760e30073c6e43277c7f0ad53cf6c083a5f4abb0220149e2bdaa4c4c02ecf93aedc9402340c84181f132cdc708361040a5fd26552e301'),x('03ddefeb041853a73fe4d78e18e28b0f8f2fd833a2de86f176f773417cfd59b3d5')))))

In [90]:
chan_point = lnmsg.ChannelPoint(funding_txid_bytes=offerReply['fundingTx'].GetTxid(),
                                output_index=offerReply['fundingOutputIdx'])

In [91]:
# our funding key
local_key = lnmsg.KeyDescriptor(
    raw_key_bytes=aliKeyDesc.raw_key_bytes,
    key_loc=lnmsg.KeyLocator(
        key_family=aliKeyDesc.key_loc.key_family,
        key_index=aliKeyDesc.key_loc.key_index
    )
)

In [92]:
premiumAmount = offer['premiumAmount']
fundAmount = offer['fundAmount']

In [93]:
chan_point_shim = lnmsg.ChanPointShim(
    amt=offer['fundAmount'] + offer['premiumAmount'],
    chan_point=chan_point,
    local_key=local_key,
    remote_key=offerReply['chanPubKey2'],
    pending_chan_id=unhexlify(offerReply['channelId']),
    thaw_height=0 # for simplicity
)

In [94]:
funding_shim = lnmsg.FundingShim(chan_point_shim=chan_point_shim)

In [95]:
open_chan_req = lnmsg.OpenChannelRequest(
    node_pubkey=unhexlify(offerReply['nodePubKey']),
    local_funding_amount=offer['fundAmount'] + offer['premiumAmount'],
    push_sat=offer['fundAmount'], # fund amount is pushed to the remote end (Bob)
    funding_shim=funding_shim
)

In [96]:
chanEventStream = ali.lnd.OpenChannel(open_chan_req)

In [97]:
next(chanEventStream)

chan_pending {
  txid: "\263\263\377\332H\027Yl\035g\320{^z8\330\226\247\331\372\351\213DC\025kDx=\260\352\270"
}
pending_chan_id: "h|\352\201\3576\316]-\355\240\220\027\033\332.s>1V\217\320N,\367MT\277\340u06"

In [98]:
# check that the channel is pending
ali.lnd.PendingChannels(lnmsg.PendingChannelsRequest())

pending_open_channels {
  channel {
    remote_node_pub: "029e06e3586e8de2e970c16d389b22c9a2d1beacdb2cd2d900472334c4a9cd8908"
    channel_point: "b8eab03d78446b1543448be9fad9a796d8387a5e7bd0671d6c591748daffb3b3:0"
    capacity: 16100000
    local_balance: 90950
    remote_balance: 16000000
    local_chan_reserve_sat: 161000
    remote_chan_reserve_sat: 161000
    initiator: INITIATOR_LOCAL
    commitment_type: STATIC_REMOTE_KEY
  }
  commit_fee: 9050
  commit_weight: 724
  fee_per_kw: 12500
}

### Sign funding tx

In [99]:
# for simplicity assume single input
assert len(aliDummyPsbtDecoded['inputs']) == 1

In [100]:
# for simplicity assume single signing key
assert len(aliDummyPsbtDecoded['inputs'][0]['bip32_derivs']) == 1

In [101]:
input_ = aliDummyPsbtDecoded['inputs'][0]
input_

{'witness_utxo': {'amount': Decimal('50.00000000'),
  'scriptPubKey': {'asm': '0 2aa244260b5f080d8b348807d9b1e28565e13dfc',
   'hex': '00142aa244260b5f080d8b348807d9b1e28565e13dfc',
   'type': 'witness_v0_keyhash',
   'address': 'bcrt1q923ygfsttuyqmze53qranv0zs4j7z00ulatthe'}},
 'non_witness_utxo': {'txid': '0c29d7d862b3b670f79d7f57eda0a512332f96e68a377535e70c14b82582001d',
  'hash': 'a0501b8219736c83b78e84b84bc9130ede6a0cb523df30f23ec9f30066b524ad',
  'version': 2,
  'size': 168,
  'vsize': 141,
  'weight': 564,
  'locktime': 0,
  'vin': [{'coinbase': '510101',
    'txinwitness': ['0000000000000000000000000000000000000000000000000000000000000000'],
    'sequence': 4294967295}],
  'vout': [{'value': Decimal('50.00000000'),
    'n': 0,
    'scriptPubKey': {'asm': '0 2aa244260b5f080d8b348807d9b1e28565e13dfc',
     'hex': '00142aa244260b5f080d8b348807d9b1e28565e13dfc',
     'reqSigs': 1,
     'type': 'witness_v0_keyhash',
     'addresses': ['bcrt1q923ygfsttuyqmze53qranv0zs4j7z00ulatthe']}

In [102]:
scriptPubKey = input_['witness_utxo']['scriptPubKey']['hex']
scriptPubKey = bs.CScript(unhexlify(scriptPubKey))

utxoAddr = bw.CBitcoinAddress.from_scriptPubKey(scriptPubKey)
utxoAddr

P2WPKHBitcoinAddress('bcrt1q923ygfsttuyqmze53qranv0zs4j7z00ulatthe')

In [103]:
# our UTXO pubkey
assert len(input_['bip32_derivs']) == 1
pubkey = input_['bip32_derivs'][0]['pubkey']
pubkey = unhexlify(pubkey)
pubkey

b'\x02\x04X-\x85\xea\xfd\xbeX\x18\x0f\x97\x9d9\xc7\xf7b\xe5\xf1\xb37\xc3\xbc\x88\x9e\xc97\xf6 \x1d\xfa\xdb\x13'

In [104]:
# create copy of other party's signed transaction
finalFundingTx = bc.CMutableTransaction(
    vin=offerReply['fundingTx'].vin,
    vout=offerReply['fundingTx'].vout
)

In [105]:
sign_desc = signmsg.SignDescriptor(
    key_desc=signmsg.KeyDescriptor(
        raw_key_bytes=pubkey
    ),
    witness_script=utxoAddr.to_redeemScript(),
    output=signmsg.TxOut(
        value=int(input_['witness_utxo']['amount'] * bc.COIN),
        pk_script=utxoAddr.to_scriptPubKey()
    ),
    sighash=bs.SIGHASH_ALL, # so that transaction cannot be altered
    input_index=len(fundingTx.vin) - 1 # assume we are the last one
)

In [106]:
signReq = signmsg.SignReq(
    raw_tx_bytes=finalFundingTx.serialize(),
    sign_descs=[sign_desc]
)

In [107]:
aliSignResp = ali.signer.SignOutputRaw(signReq)
aliSignResp

raw_sigs: "0E\002!\000\340\260\271\374\035SQ$p\303\367\035\357u\233\315r\tg\316*t\271m\273t\254dg\0223`\002 hW\030\020\013\340z\357\221P\214V\363\006 \252\361\014d\301\345+\313%\224\316ce\341\001^\212"

In [108]:
signature = aliSignResp.raw_sigs[0] + bytes([bs.SIGHASH_ALL])

In [109]:
witness = [signature, pubkey]
witness = bc.CTxInWitness(bs.CScriptWitness(witness))
witness

CTxInWitness(CScriptWitness(x('3045022100e0b0b9fc1d53512470c3f71def759bcd720967ce2a74b96dbb74ac64671233600220685718100be07aef91508c56f30620aaf10c64c1e52bcb2594ce6365e1015e8a01'),x('0204582d85eafdbe58180f979d39c7f762e5f1b337c3bc889ec937f6201dfadb13')))

In [110]:
# combine signatures
finalFundingTx.wit = bc.CTxWitness([offerReply['fundingTx'].wit.vtxinwit[0], witness])

### Submit funding transaction

In [111]:
fundingTxId2 = brpc.sendrawtransaction(finalFundingTx)
fundingTxId2

b'\xb3\xb3\xff\xdaH\x17Yl\x1dg\xd0{^z8\xd8\x96\xa7\xd9\xfa\xe9\x8bDC\x15kDx=\xb0\xea\xb8'

In [112]:
assert fundingTxId2 == fundingTxId

In [113]:
_ = list(brpc.generatetoaddress(6, minerAddr))

### Check that the channel has been opened

In [114]:
# should be no pending channels
ali.lnd.PendingChannels(lnmsg.PendingChannelsRequest())

In [115]:
ali.lnd.ListChannels(lnmsg.ListChannelsRequest())

channels {
  active: true
  remote_pubkey: "029e06e3586e8de2e970c16d389b22c9a2d1beacdb2cd2d900472334c4a9cd8908"
  channel_point: "b8eab03d78446b1543448be9fad9a796d8387a5e7bd0671d6c591748daffb3b3:0"
  chan_id: 124244814004224
  capacity: 16100000
  local_balance: 90950
  remote_balance: 16000000
  commit_fee: 9050
  commit_weight: 724
  fee_per_kw: 12500
  csv_delay: 1934
  initiator: true
  chan_status_flags: "ChanStatusDefault"
  local_chan_reserve_sat: 161000
  remote_chan_reserve_sat: 161000
  static_remote_key: true
  lifetime: 27
  uptime: 27
  commitment_type: STATIC_REMOTE_KEY
  push_amount_sat: 16000000
  local_constraints {
    csv_delay: 1934
    chan_reserve_sat: 161000
    dust_limit_sat: 573
    max_pending_amt_msat: 15939000000
    min_htlc_msat: 1
    max_accepted_htlcs: 483
  }
  remote_constraints {
    csv_delay: 1934
    chan_reserve_sat: 161000
    dust_limit_sat: 573
    max_pending_amt_msat: 15939000000
    min_htlc_msat: 1
    max_accepted_htlcs: 483
  }
}